In [244]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import re

In [245]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [246]:
LOCAL = False

if LOCAL:
    local_suffix = "_local"
else:
    local_suffix = ""

In [247]:
%%capture pwd
!pwd

In [248]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "articles/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
height = config["height"]
width = config["width"]
uid, height, width

('how-have-economists-thought-about-the-environment', 300, 500)

# Fig 1

In [249]:
df = pd.read_excel("raw/World_Population_and_oil_prices.xlsx", sheet_name="Population")
df.columns = ["year", "population", "population_growth", "population2"]
df = df.drop("population2", axis=1)
df["population_growth"] /= 100.0
df["population"] *= 1000
df

,year,population,population_growth
0,1870,1.271810e+09,NaN
1,1871,1.280462e+09,0.006802
2,1872,1.289172e+09,0.006802
3,1873,1.297941e+09,0.006802
4,1874,1.306771e+09,0.006802
...,...,...,...
146,2016,7.424485e+09,0.011637
147,2017,7.509410e+09,0.011439
148,2018,7.592476e+09,0.011062
149,2019,7.673345e+09,0.010651


In [250]:
f = "fig1_population"
f1 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f1.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f1 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,population,population_growth
0,1870,1.271810e+09,NaN
1,1871,1.280462e+09,0.006802
2,1872,1.289172e+09,0.006802
3,1873,1.297941e+09,0.006802
4,1874,1.306771e+09,0.006802


In [251]:
base = alt.Chart(f1).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-turquiose"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-turquiose"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "population:Q",
        axis=alt.Axis(
            grid=False,
            title="Population",
            titleX=-10,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            orient="right",
            format="~s",
            titleAlign="right",
            labelColor=colors["eco-turquiose"],
            titleColor=colors["eco-turquiose"],
            tickColor=colors["eco-turquiose"],
            domainColor=colors["eco-turquiose"],
        ),
    )
)
line2 = base.mark_line(color=colors["eco-dot"]).encode(
    y=alt.Y(
        "population_growth:Q",
        axis=alt.Axis(
            grid=False,
            title="Population growth",
            titleX=10,
            titleY=5,
            titleBaseline="bottom",
            titleAngle=0,
            orient="left",
            format=".1%",
            titleAlign="left",
            labelColor=colors["eco-dot"],
            titleColor=colors["eco-dot"],
            tickColor=colors["eco-dot"],
            domainColor=colors["eco-dot"],
        ),
    )
)
layer1 = (
    ((line1 + line2).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
    .resolve_scale(y="independent")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "w").write(readme)
layer1

alt.LayerChart(...)

# Fig 2

In [252]:
df = pd.read_csv("raw/wpd_datasets.csv", skiprows=1)
dfs = []
for i in range(0, 10, 2):
    dfs.append(df[df.columns[i : i + 2]])
variables = [
    "Resources",
    "Population",
    "Food (per capita)",
    "Industrial output (per capita)",
    "Pollution",
]
for i in range(len(dfs)):
    dfs[i].columns = ["year", "value"]
    dfs[i]["variable"] = variables[i]
df = pd.concat(dfs).dropna().sort_values(by="year")
df["year"] = df["year"].astype(int)
df = df.groupby(["variable", "year"]).mean().reset_index()
df = (
    pd.DataFrame(range(1900, 2100), columns=["year"])
    .join(df.set_index(["year", "variable"]).unstack()["value"], on="year")
    .set_index("year")
    .interpolate()
    .bfill()
)
df = df.rolling(3).mean()
df = df.stack().reset_index()
df.columns = ["year", "variable", "value"]
df["value"] = [min(1, max(0, i)) for i in df["value"]]

<ipython-input-252-be2d4e0611f9>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]["variable"] = variables[i]


In [253]:
f = "fig2_world3"
f2 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f2.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f2 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,variable,value
0,1902,Food (per capita),0.204385
1,1902,Industrial output (per capita),0.045625
2,1902,Pollution,0.002397
3,1902,Population,0.114651
4,1902,Resources,0.999365


In [254]:
base = alt.Chart(f2).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=3,
            titleFontSize=10,
            titleFontWeight="normal",
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    )
)
line = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.05,
    line={"color": colors["eco-turquiose"]},
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=25,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".1f",
        ),
    ),
    color=alt.Color(
        "variable:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-mid-blue"],
                colors["eco-light-blue"],
                colors["eco-green"],
                colors["eco-gray"],
            ]
        ),
    ),
)
labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="variable:N")
    .transform_filter("datum.year>2098")
    .transform_filter("datum.variable!='Pollution'")
)
labels2 = (
    line.mark_text(align="left", dx=5, dy=-10)
    .encode(text="variable:N")
    .transform_filter("datum.year>2098")
    .transform_filter("datum.variable=='Pollution'")
)
layer1 = ((line + labels + labels2).properties(height=300, width=400)).configure_view(
    stroke=None
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 3

In [255]:
df = pd.read_csv("raw/wpd_datasets2.csv", skiprows=1)
dfs = []
for i in range(0, 10, 2):
    dfs.append(df[df.columns[i : i + 2]])
variables = [
    "Resources",
    "Population",
    "Food (per capita)",
    "Industrial output (per capita)",
    "Pollution",
]
for i in range(len(dfs)):
    dfs[i].columns = ["year", "value"]
    dfs[i]["variable"] = variables[i]
df = pd.concat(dfs).dropna().sort_values(by="year")
df["year"] = df["year"].astype(int)
df = df.groupby(["variable", "year"]).mean().reset_index()
df = (
    pd.DataFrame(range(1900, 2100), columns=["year"])
    .join(df.set_index(["year", "variable"]).unstack()["value"], on="year")
    .set_index("year")
    .interpolate()
    .bfill()
)
df = df.rolling(3).mean()
df = df.stack().reset_index()
df.columns = ["year", "variable", "value"]
df = df[~((df["variable"] == "Pollution") & (df["year"] > 2042))]
df["value"] = [min(1, max(0, i)) for i in df["value"]]

<ipython-input-255-4f51a56084d1>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]["variable"] = variables[i]


In [256]:
f = "fig3_world3_double"
f3 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f3.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f3 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,variable,value
0,1902,Food (per capita),0.206651
1,1902,Industrial output (per capita),0.033922
2,1902,Pollution,0.000000
3,1902,Population,0.110451
4,1902,Resources,1.000000


In [257]:
base = alt.Chart(f3).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="right",
            titleAnchor="end",
            title="",
            titleY=-15,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=3,
            titleFontSize=10,
            titleFontWeight="normal",
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    )
)
line = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.05,
    line={"color": colors["eco-turquiose"]},
).encode(
    y=alt.Y(
        "value:Q",
        axis=alt.Axis(
            grid=True,
            title="",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=25,
            titleY=25,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".1f",
        ),
    ),
    color=alt.Color(
        "variable:N",
        legend=None,
        scale=alt.Scale(
            range=[
                colors["eco-turquiose"],
                colors["eco-mid-blue"],
                colors["eco-light-blue"],
                colors["eco-green"],
                colors["eco-gray"],
            ]
        ),
    ),
)
labels = (
    line.mark_text(align="left", dx=5)
    .encode(text="variable:N")
    .transform_filter("datum.year>2098")
    .transform_filter("datum.variable!='Industrial output (per capita)'")
    .transform_filter("datum.variable!='Pollution'")
)
labels2 = (
    line.mark_text(align="left", dx=5, dy=10)
    .encode(text="variable:N")
    .transform_filter("datum.year>2098")
    .transform_filter("datum.variable=='Industrial output (per capita)'")
)
labels3 = (
    line.mark_text(align="left", dx=5, dy=10)
    .encode(text="variable:N")
    .transform_filter("datum.year==2042")
    .transform_filter("datum.variable=='Pollution'")
)
layer1 = (
    (line + labels + labels2 + labels3).properties(height=300, width=400)
).configure_view(stroke=None)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 4

In [258]:
df = pd.read_excel(
    "raw/World_Population_and_oil_prices.xlsx", sheet_name="Oil price index"
)
df = df[["Crude oil price (US dollars per barrel)", "Index (1990-2000 = 1).1"]].loc[1:]
df.columns = ["year", "normalized_2020USD_price_1990_2000_100"]

In [259]:
f = "fig4_oil_price"
f4 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f4.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f4 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,normalized_2020USD_price_1990_2000_100
1,1861.0,42.555433
2,1862.0,82.071193
3,1863.0,199.63263
4,1864.0,402.123898
5,1865.0,335.931155


In [260]:
base = alt.Chart(f4).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.7,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "normalized_2020USD_price_1990_2000_100:Q",
        axis=alt.Axis(
            grid=True,
            title="Index of crude oil prices in 2000 US dollars (average of 1990-2000 = 100) ",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            #             labelAlign="left",
            #             labelBaseline="middle",
            #             labelPadding=-5,
            #             labelOffset=-10,
            titleX=5,
            titleY=25,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
average = (
    alt.Chart(pd.DataFrame([{"year": 1990, "y": 100}, {"year": 2000, "y": 100}]))
    .mark_line(color=colors["eco-dot"])
    .encode(x="year:Q", y="y:Q")
)
layer1 = (
    ((line1 + average).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)

# Fig 5

In [261]:
df = pd.read_excel(
    "raw/World_Population_and_oil_prices.xlsx", sheet_name="CO2 and population"
)
df = df[["Million tonnes of carbon dioxide", "CO2 emissions ", "Global Population"]]
df.columns = ["year", "co2_emissions_1990_100", "population_1990_100"]

In [262]:
f = "fig5_emissions_population"
f5 = eco_git_path + f + ".csv"
df.to_csv("data/" + f + ".csv")
f += local_suffix
open("visualisation/" + f + ".html", "w").write(
    vega_embed.replace(
        "JSON_PATH", f5.replace("/data/", "/visualisation/").replace(".csv", ".json")
    )
)
if LOCAL:
    f5 = df
readme = "### " + f + '\n!["' + f + '"](visualisation/' + f + '.png "' + f + '")\n\n'
df.head()

,year,co2_emissions_1990_100,population_1990_100
0,1965,51.927039,62.917004
1,1966,54.271398,64.243714
2,1967,55.945045,65.560407
3,1968,58.942567,66.908185
4,1969,62.572518,68.305346


In [263]:
base = alt.Chart(f5).encode(
    x=alt.X(
        "year:Q",
        axis=alt.Axis(
            grid=False,
            titleAlign="center",
            titleAnchor="middle",
            title="",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            orient="bottom",
            labelAngle=0,
            format=".0f",
        ),
    )
)
line1 = base.mark_area(
    interpolate="monotone",
    fillOpacity=0.4,
    line={"color": colors["eco-mid-blue"]},
    color=alt.Gradient(
        gradient="linear",
        stops=[
            alt.GradientStop(color="white", offset=0),
            alt.GradientStop(color=colors["eco-mid-blue"], offset=1),
        ],
        x1=1,
        x2=1,
        y1=0.9,
        y2=0,
    ),
).encode(
    y=alt.Y(
        "co2_emissions_1990_100:Q",
        axis=alt.Axis(
            grid=True,
            title="relative to 1990 = 100",
            titleAnchor="start",
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            titleFontSize=10,
            titleFontWeight="normal",
            ticks=False,
            labelAlign="left",
            labelBaseline="middle",
            labelPadding=-5,
            labelOffset=-10,
            titleX=30,
            titleY=13,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            tickCount=4,
            format=".0f",
        ),
    ),
)
line2 = base.mark_line(color=colors["eco-turquiose"]).encode(
    y=alt.Y("population_1990_100:Q")
)
average = (
    alt.Chart(pd.DataFrame([{"year": 1990, "y": 100}]))
    .mark_point(color=colors["eco-dot"])
    .encode(x="year:Q", y="y:Q")
)
labels = (
    alt.Chart(
        pd.DataFrame(
            [
                {"year": 1995, "y": 120, "t": "Population"},
                {"year": 1999, "y": 140, "t": "CO₂ emissions"},
            ]
        )
    )
    .mark_text()
    .encode(x="year:Q", y="y:Q",text='t:N',color=alt.Color('t:N',legend=None,
                                                           scale=alt.Scale(range=[colors["eco-mid-blue"],colors["eco-turquiose"]])))
)
layer1 = (
    ((line1 + line2 + average + labels).properties(height=300, width=400))
    .configure_view(stroke=None)
    .properties(title="")
)
layer1.save("visualisation/" + f + ".json")
layer1.save("visualisation/" + f + ".png", scale_factor=2.0)
layer1.save("visualisation/" + f + ".svg")
open("README.md", "a").write(readme)
layer1

alt.LayerChart(...)